In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用TF Serving容器进行在线预测的自定义训练文本二分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_online_tfserving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_online_tfserving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署用于在线预测的自定义文本二元分类模型，使用TF Serving容器进行预测。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[IMDB电影评论](https://www.tensorflow.org/datasets/catalog/imdb_reviews)。在本教程中使用的数据集版本存储在一个公共的云存储桶中。训练好的模型能够预测评论是积极的还是消极的情感。

### 目标

在本教程中，您将使用Vertex客户端库，在Google预构建的Docker容器中的Python脚本中创建一个自定义模型，然后使用TF Serving容器对部署的模型进行预测。您也可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建自定义模型。

执行的步骤包括：

- 创建用于训练模型的Vertex自定义作业。
- 创建一个`TrainingPipeline`资源。
- 训练一个TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 将模型上传为Vertex `Model`资源。
- 使用TF Serving容器将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

### 成本

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

也安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和谷歌*云存储*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300 的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。 以下是Vertex支持的区域。 我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来进行Vertex的训练。 并非所有区域都支持所有Vertex服务。 有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到本教程中将要创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**将此服务帐号授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，选择**Vertex管理员**。再次输入“Storage Object Admin”，选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐号密钥路径填写到GOOGLE_APPLICATION_CREDENTIALS变量中，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶。Vertex 将从这个包中运行代码。在本教程中，Vertex 还会将您的作业结果产生的训练模型保存在同一个桶中。然后，您可以根据这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google 云项目中全局唯一，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚未存在时才能执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶的内容验证访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：数据集、模型、作业、管道和端点服务的顶点API服务端点。
- `PARENT`：数据集、模型、作业、管道和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

### 安装 Docker（Colab 或本地）

如果您正在使用 Google Cloud Notebook，则Docker 已经安装。跳过这些步骤。

默认情况下，Colab 上没有安装 Docker。如果您正在运行 colab，则需要按照以下步骤安装 Docker。

In [ ]:
if "google.colab" in sys.modules:
    ! sudo apt update
    ! sudo apt install apt-transport-https ca-certificates curl software-properties-common
    ! curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
    ! sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
    ! sudo apt update
    ! sudo apt install docker-ce

启动 Docker 服务

启动 `docker` 服务。

In [ ]:
if "google.colab" in sys.modules:
    ! sudo service docker start

#### 硬件加速器

设置用于训练和预测的硬件加速器（如GPU）。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，并指定分配给虚拟机（VM）实例的GPU数量。例如，要使用带有4个Nvidia Telsa K80 GPU的GPU容器映像分配给每个VM，您应该指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以在CPU上运行容器映像。

*注意*：TF 2.3之前的GPU支持可能无法加载此教程中的定制模型。这是一个已知问题，在TF 2.3中已修复 -- 这是由于在服务函数中生成的静态图操作。如果在您自己的定制模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）映像

接下来，我们将为训练设置Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下是一些可用的预构建映像的列表：

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
 - TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

获取最新列表，请查看[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 用于预测的容器（Docker）镜像

接下来，您需要设置用于预测的TF Serving Docker容器镜像。

1. 从Docker Hub拉取适用于TF Serving的相应CPU或GPU Docker镜像。
2. 为图像创建标签，注册到云容器注册表（gcr.io）。
3. 将图像注册到云容器注册表。

有关使用TF Serving容器的更多详细信息，请参阅[TF Serving](https://www.tensorflow.org/tfx/serving/docker)。

In [ ]:
if DEPLOY_GPU:
    ! docker pull tensorflow/serving:latest-gpu
    DEPLOY_IMAGE = "gcr.io/" + PROJECT_ID + "/tf_serving:gpu"
else:
    ! docker pull tensorflow/serving:latest
    DEPLOY_IMAGE = "gcr.io/" + PROJECT_ID + "/tf_serving"

! docker tag tensorflow/serving $DEPLOY_IMAGE
! docker push $DEPLOY_IMAGE

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU有3.75GB内存。
     - `n1-highmem`：每个vCPU有6.5GB内存
     - `n1-highcpu`：每个vCPU有0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以在训练和部署中使用n2和e2机器类型，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

教程

现在，您已准备好开始为IMDB电影评论创建自己的自定义模型并训练。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在你的一侧（Python脚本）上，你将创建一个客户端向Vertex服务器发送请求并接收响应。

在本教程中，你将为工作流程的不同步骤使用不同的客户端。因此，请提前设置它们。

- 为`Model`资源设置模型服务。
- 为部署设置端点服务。
- 为批处理作业和自定义训练设置作业服务。
- 为提供服务设置预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

有两种方法可以使用容器图像来训练一个自定义模型：

- **使用Google Cloud预构建的容器**。如果您使用预构建的容器，您还需要指定一个要安装到容器图像中的Python软件包。这个Python软件包包含了您训练自定义模型的代码。

- **使用您自己的自定义容器图像**。如果您使用您自己的容器，容器需要包含您训练自定义模型的代码。

## 准备您的自定义工作规范

现在您的客户已经准备好了，您的第一步是为您的自定义培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：指定用于培训的机器类型和数量（单个或分布式）
- `python_package_spec`：要与预构建容器一起安装的Python包规范。

### 准备您的机器规格

现在为您的自定义训练作业定义机器规格。这告诉 Vertex 需要为训练提供什么类型的机器实例。
  - `machine_type`: 要提供的 GCP 实例类型 -- 例如，n1-standard-8。
  - `accelerator_type`: 如果有的话，硬件加速器的类型。在本教程中，如果您之前设置过变量`TRAIN_GPU != None`，则表示您正在使用 GPU；否则您将使用 CPU。
  - `accelerator_count`: 加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉Vertex应该为每台训练机器实例配置什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小，以GB为单位。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作池规范

接下来，您需要定义自定义训练作业的工作池规范。工作池规范将包括以下内容：

- `replica_count`：要创建的此机器类型的实例数。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的docker镜像。

- `package_uris`：这是要安装在预配实例上的Python训练包的位置（URIs）列表。这些位置需要在Cloud Storage存储桶中。它们可以是单独的Python文件，也可以是整个包的zip（归档）。在后一种情况下，作业服务将把内容解压缩到docker镜像中。

- `python_module`：要调用以运行自定义训练作业的Python模块（脚本）。在此示例中，您将调用`trainer.task.py` - 请注意，不需要附加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在此示例中，您将设置：
   - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。有两种方法可以告诉训练脚本保存模型工件的位置：
      - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
   - `"--epochs=" + EPOCHS`：训练的时代数。
   - `"--steps=" + STEPS`：每个时代的步骤（批次）数。
   - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分发策略。
      - `"single"`：单个设备。
      - `"mirror"`：单个计算实例上的所有GPU设备。
      - `"multi"`：所有计算实例上的所有GPU设备。

*注意*：对于TF Serving，`MODEL_DIR`必须以数字为子文件夹结尾，例如，1。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}/1".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_imdb.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装定制工作规范的完整描述：

- `display_name`：您为此自定义工作分配的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务Cloud Storage位置保存模型构件的位置（当变量`DIRECT = False`时）。 然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径形式为：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为定制培训任务组装一个Python包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明书。

文件`trainer/task.py`是执行定制培训任务的Python脚本。*注意*，在工作池规格中提到它时，我们将目录斜杠替换为点（`trainer.task`），并且去掉了文件后缀（`.py`）。

#### 包装

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: IMDB Movie Reviews text binary classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您可以编写训练脚本task.py的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`获取。
- 从TF数据集（tfds）加载IMDB电影评论数据集。
- 使用TF.Keras模型API构建简单的RNN模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 使用由`args.epochs`指定的时期训练模型（`fit()`）。
- 将训练好的模型（`save(args.model_dir)`）保存到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for IMDB

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets():
    dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    encoder = info.features['text'].encoder

    padded_shapes = ([None],())
    return train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes), encoder


train_dataset, encoder = make_datasets()

# Build the Keras model
def build_and_compile_rnn_model(encoder):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model


with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_rnn_model(encoder)

# Train the model
model.fit(train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

请将培训脚本存储到您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar球，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_imdb.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，带有以下参数：

- `parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`：自定义作业的规范。

您将显示`response`对象中返回的一些字段，其中最感兴趣的两个是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

### 获取自定义作业的信息

接下来，请使用这个帮助函数 `get_custom_job`，它接受以下参数：

- `name`：自定义作业的Vertex完全限定标识符。

帮助函数调用作业客户端服务的`get_custom_job`方法，其中包含以下参数：

- `name`：自定义作业的Vertex完全限定标识符。

如果你记得，在调用`create_custom_job`方法时，你在`response.name`字段中获得了自定义作业的Vertex完全限定标识符，并将该标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

部署

训练上述模型可能需要20分钟以上的时间。

一旦您完成模型的训练，您可以通过将`end_time`减去`start_time`来计算训练模型所花费的实际时间。对于您的模型，我们需要知道保存模型的位置，Python脚本将其保存在您本地云存储桶的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在一个云存储桶中。现在从云存储桶中加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的云存储路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们来看一下模型的表现如何。

### 加载评估数据

您将使用`tfds.datasets`的`load()`方法加载IMDB电影评论测试（留存）数据。这将返回一个包含两个元素的元组，第一个元素是数据集，第二个是数据集的信息，其中包含预定义的词汇编码器。编码器将单词转换为数字嵌入，这是预先训练的并在自定义训练脚本中使用的。

在训练模型时，您需要为文本设置一个固定的输入长度。对于正向馈送批处理，相应的`tf.dataset`的`padded_batch()`属性被设置为将每个输入序列填充到相同的形状以进行批处理。

对于测试数据，您还需要相应地设置`padded_batch()`属性。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]
encoder = info.features["text"].encoder

BATCH_SIZE = 64
padded_shapes = ([None], ())
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes)

### 执行模型评估

现在评估一下自定义作业中模型的表现如何。

In [ ]:
model.evaluate(test_dataset)

上传模型用于服务

接下来，您将从自定义作业中将您的TF.Keras模型上传到Vertex`Model`服务，这将为您的自定义模型创建一个Vertex`Model`资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型期望的格式。如果您将编码的数据发送到Vertex，您的服务函数会确保在将数据作为输入传递到模型之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，该请求将被HTTP服务器接收。HTTP服务器从HTTP请求内容正文中提取预测请求。提取的预测请求将转发到服务函数。对于谷歌预构建的预测容器，请求内容将作为`tf.string`传递到服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作JSON对象等。

预处理函数和后处理函数都会转换为与模型融合的静态图。从底层模型输出的结果将传递给后处理函数。后处理函数将转换/打包好的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

建立TF.Keras模型的服务函数时需要考虑的一个问题是，它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果这样做，您将在服务函数的编译过程中收到一个错误，指示您正在使用不受支持的EagerTensor。

获取服务功能签名

通过重新加载模型到内存中，并查询每个层对应的签名，您可以获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要了解服务功能的输入层名称--在稍后进行预测请求时将使用该名称。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

###上传模型

使用这个帮助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这个步骤，您就可以像任何其他Vertex `Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

该帮助函数接受以下参数：

- `display_name`：`Endpoint`服务的人类可读名称。
- `image_uri`：用于模型部署的容器映像。
- `model_uri`：我们的SavedModel工件在云存储中的路径。对于本教程，这是`trainer/task.py`保存模型工件的云存储位置，我们在变量`MODEL_DIR`中指定。

该帮助函数调用`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`, `Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的人类可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，您将保持此字段为空（`''`）。
- `artificat_uri`：模型以SavedModel格式存储的云存储路径。
- `container_spec`：这是将安装在端点上的Docker容器的规范，从中模型将提供预测。在这个教程中，您使用TF Serving作为自定义预测容器。由于这是一个自定义容器，您需要告诉Vertex如何启动并与服务二进制通信：
    - `command`：要启动的服务二进制（HTTP服务器）。
    - `args`：传递给服务二进制的参数。对于TF Serving，所需的参数是：
        - `--model_name`：分配给模型的人类可读名称。
        - `--model_base_name`：在容器中存储模型工件的位置。Vertex服务将变量$(AIP_STORAGE_URI)设置为在容器中安装模型工件的位置。
        - `--rest_api_port`：发送基于REST的预测请求的端口。可以是8080或8501（TF Serving的默认值）。
        - `--port`：发送基于gRPC的预测请求的端口。对于TF Serving，应为8500。

    - `health_route`：服务定期ping以获取响应以验证服务二进制是否运行的URL。对于TF Serving，这将是/v1/models/[model_name]。
    - `predict_route`：将REST基础预测请求路由到的服务的URL。对于TF Serving，这将是/v1/models/[model_name]:predict。

将模型上传到Vertex模型资源将返回一个长时间运行的操作，因为可能需要一些时间。您可以调用response.result()，这是一个同步调用，当Vertex模型资源准备就绪时将返回。

帮助函数返回对应的Vertex模型实例upload_model_response.model的Vertex完全合格标识符。您将保存该标识符以备后续步骤中的变量model_to_deploy_id使用。

*注意*：您需要删除模型路径中的末尾数字子文件夹（例如，/1）以进行上传。Vertex服务将上传包含模型工件的子文件夹上面的父文件夹 -- 这是TensorFlow服务二进制文件所期望的。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE
MODEL_NAME = "imdb-" + TIMESTAMP


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": ["/usr/bin/tensorflow_model_server"],
            "args": [
                "--model_name=" + MODEL_NAME,
                "--model_base_path=" + "$(AIP_STORAGE_URI)",
                "--rest_api_port=8080",
                "--port=8500",
                "--file_system_poll_wait_seconds=31540000",
            ],
            "health_route": "/v1/models/" + MODEL_NAME,
            "predict_route": "/v1/models/" + MODEL_NAME + ":predict",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "imdb-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy[:-2]
)

获取`Model`资源信息

现在让我们获取您的模型的模型信息。使用这个帮助函数`get_model`，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个帮助函数调用了Vertex `Model`客户端服务的`get_model`方法，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署训练好的Vertex自定义`Model`资源。这需要两个步骤：

1. 创建一个`Endpoint`资源，用于部署`Model`资源。

2. 将`Model`资源部署到`Endpoint`资源中。

### 创建 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个终端节点，用于部署模型以提供预测服务，参数如下：

- `display_name`：`Endpoint` 资源的可读名称。

这个辅助函数使用终端节点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`：`Endpoint` 资源的可读名称。

创建 `Endpoint` 资源将返回一个长时间运行的操作，因为可能需要一些时间来为服务创建 `Endpoint` 资源。您可以调用 `response.result()`，这是一个同步调用，当终端节点资源就绪时将返回。这个辅助函数返回 `Endpoint` 资源的 Vertex 完全合格的标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "imdb_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“Endpoint”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例的扩缩容

处理在线预测请求时，您可以选择以下几种计算实例的扩缩容方式：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设为一。

- 手动扩缩容：在线预测请求在您手动指定的固定数量的计算实例上进行分割处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，将提供固定数量的计算实例，并将在线预测请求均匀分配到这些实例上。

- 自动扩缩容：在线预测请求在可扩展数量的计算实例上进行分割处理。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时需要提供和回收的实例数量，并根据负载条件将最大（`MAX_NODES`）数量的计算实例设置为需要提供的数量。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来将从训练管道中上传（部署）的 `Model` 资源部署到为提供预测而创建的 `Endpoint` 资源，使用以下参数：

- `model`: 要上传（部署）的模型的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读的名称。
- `endpoint`: 要将模型部署到的 Vertex 完全限定的端点标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 流量分配给该模型的端点的百分比，指定为一个或多个键/值对的字典。
  - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是上传的这个模型，100 表示 100% 的流量。
  - 如果端点上有已存在的模型，流量将被分配给这些模型，则使用 `model_id` 来指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是已部署到端点的现有模型的模型标识符。这些百分比必须相加为 100。

现在让我们更深入地了解 `deployed_model` 参数。该参数被指定为一个具有最少要求字段的 Python 字典：

- `model`: 要部署的（上传的）模型的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读的名称。
- `disable_container_logging`: 这禁用容器事件的日志记录，例如执行失败（默认情况下容器日志记录是启用的）。在调试部署时通常启用容器日志记录，在生产环境部署后禁用。
- `dedicated_resources`: 指定为提供用于提供预测请求的计算实例（副本）的数量。
  - `machine_spec`: 要提供的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`: 要首次提供的计算实例数，你之前将其设置为变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的计算实例的最大数量，你之前将其设置为变量 `MAX_NODES`。

#### 流量分配

现在让我们更深入地了解 `traffic_split` 参数。该参数被指定为一个Python字典。这可能会有点令人困惑。让我解释一下，你可以将多个模型实例部署到一个端点，然后设置每个实例接收多少流量（百分比）。

为什么要这样做？也许你已经在生产中部署了先前版本 -- 我们将其称为 v1。你在 v2 上获得了更好的模型评估，但在将其部署到生产环境之前，你并不确定它是否真的更好。因此，在流量分配的情况下，你可能希望将 v2 部署到与 v1 相同的端点，但它只得到了比如 10% 的流量。这样，你可以在不干扰大多数用户的情况下监视它的表现，直到最终作出决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将同步等待操作完成，调用 `response.result()`，这将阻塞直到模型部署完成。如果这是首次将模型部署到端点，则可能需要额外的几分钟来完成资源的配置。

In [ ]:
DEPLOYED_NAME = "imdb_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出在线预测请求

现在对部署的模型进行在线预测。

### 准备请求内容

由于数据集是一个 `tf.dataset`，它充当生成器的角色，我们必须将其用作迭代器来访问测试数据中的数据项。 我们执行以下操作以从测试数据中获取单个数据项：

- 使用 `take(1)` 方法将每次迭代绘制的批次数属性设置为一。
- 通过测试数据进行一次迭代 - 即，在 for 循环中执行中断。
- 在单次迭代中保存数据项，其形式为元组。
- 数据项将是元组的第一个元素，然后您将其从张量转换为 numpy 数组 - `data[0].numpy()`。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]

test_dataset.take(1)
for data in test_dataset:
    print(data)
    break

test_item = data[0].numpy()

发送预测请求

好的，现在您有一个测试数据项目。使用这个辅助函数`predict_data`，它接受以下参数：

- `data`：测试数据项目是一个64填充的numpy 1D数组。
- `endpoint`：已部署模型的Vertex AI完全限定标识符。
- `parameters_dict`：用于提供服务的其他参数。

此函数使用预测客户端服务，并调用`predict`方法，其中包含以下参数：

- `endpoint`：已部署模型的Vertex AI完全限定标识符。
- `instances`：要预测的实例（数据项目）列表。
- `parameters`：用于提供服务的其他参数。

要将测试数据传递给预测服务，您必须将其打包以传输到服务二进制文件，如下所示：

    1. 将数据项目从1D numpy数组转换为1D Python列表。
    2. 将预测请求转换为序列化的Google协议缓冲区（`json_format.ParseDict()`）

预测请求中的每个实例都是以下形式的字典条目：

                        {input_name: content}

- `input_name`：底层模型的输入层的名称。
- `content`：作为1D Python列表的数据项目。

由于`predict()`服务可以接受多个数据项目（实例），您将将单个数据项目作为一个数据项目列表发送。最后一步，您将实例列表打包成Google的协议缓冲区格式--这是我们传递给`predict()`服务的内容。

`response`对象返回一个列表，列表中的每个元素对应请求中的相应图像。您将在每个预测的输出中看到：

- `predictions`--在0（负面）和1（正面）之间的预测二进制情感。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

## 下线`Model`资源

现在从提供服务的`Endpoint`资源中下线您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：当`Model`资源部署到`Endpoint`服务时由终端服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的 Vertex 全限定标识符。

此函数调用终端客户端服务的`undeploy_model`方法，其中包含以下参数：

- `deployed_model_id`：当`Model`资源部署时由终端服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的 Vertex 全限定标识符。
- `traffic_split`：在`Endpoint`资源上的其余部署模型之间如何分配流量。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将`traffic_split`设置为空对象{}来使其保持空白。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME